# Yolo8 Training Scripts

conda info --envs

conda create --name env1 python==3.12.7

conda activate env1

In [ ]:
# !pip install pydantic==1.10.7
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
# !pip install ultralytics
# !pip install opencv-python-headless
# !pip install matplotlib
# !pip install tensorboard torch-tb-profiler

# !pip show torch torchvision
# !nvcc --version

### 1. Dataset folder structure.

dataset/
<br>│
<br>├── test/
<br>│   ├── images/
<br>│   ├── labels/
<br>│
<br>├── train/
<br>│   ├── images/
<br>│   ├── labels/
<br>│
<br>├── valid/
<br>│   ├── images/
<br>│   ├── labels/

<br>

### 2. Config file

data.yaml

<br>train: ../train/images
<br>val: ../valid/images
<br>test: ../test/images
<br>
<br>nc: 3
<br>names: ['dear', 'monkey', 'person']


In [1]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.pt')  # Choose a YOLOv8 model variant (e.g., 'yolov8n.pt', 'yolov8s.pt')

# Train the model with TensorBoard logging enabled
model.train(
    data=r'C:\Users\winai\All Data\My AI Data\Training Day 6\Data\monkey-in-a-zoo.v2\data.yaml',
    epochs=100,
    batch=16,
    imgsz=640,
    # device=0,
    project=r'C:\Users\winai\All Data\My AI Data\Training Day 6\Data\monkey-in-a-zoo.v2\runs',  # Directory where TensorBoard logs will be saved
    name='yolov8_experiment',  # Sub-directory for this experiment
    exist_ok=True  # Overwrites the logs if already exist
)

      1/100         0G      1.542      2.107      1.186        177        640: 100%|██████████| 53/53 [05:28<00:00,  6.20s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:13<00:00,  4.38s/it]

                   all         81        861      0.978      0.269      0.631      0.413



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      2/100         0G      1.508      1.316      1.151        294        640:   9%|▉         | 5/53 [00:38<06:06,  7.64s/it]


KeyboardInterrupt: 

# Inference

In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random

classes_map = {}
classes_map[0] = "dear"
classes_map[1] = "monkey"
classes_map[2] = "person"

# Load a model
model = YOLO('C:/Users/winai/All Data/My AI Data/Training Day 6/Data/monkey-in-a-zoo/runs/yolov8_experiment/weights/best.pt')

obj_classes = list(model.names.values())
classes_ids = [obj_classes.index(clas) for clas in obj_classes]
_colors = [random.choices(range(256), k=3) for _ in classes_ids]

# 1. Create models directory
L_PATH = 'C:/Users/winai/All Data/My AI Data/Training Day 6/Data/monkey-in-a-zoo/test/images/'

imgs = [
    L_PATH+'f_106_jpg.rf.47444b89570eda8386a6e6c9c883d58f.jpg',
    L_PATH+'f_201_jpg.rf.c9ae6f0de78315c8c329d0d6bac87120.jpg',
    L_PATH+'f_331_jpg.rf.b14802de64f78c6d8f55c88d7df449f0.jpg',
    L_PATH+'f_345_jpg.rf.594cfdd10607635bbd226273246cba0e.jpg',
    L_PATH+'f_247_jpg.rf.227a16eedbe119b6b06908eea8240948.jpg',
]

for img in imgs:
    frame = cv2.imread(img)

    results = model.predict(frame, conf=0.2, show_labels=True, vid_stride=3, verbose=False)
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = box.xyxy.squeeze().tolist()
            confidence = box.conf.squeeze().tolist()
            color_number = classes_ids.index(int(box.cls[0]))
            label = classes_map[color_number]
            color = _colors[color_number]
            cv2.rectangle(frame, (int(x1),int(y1)), (int(x2),int(y2)),color,2)
            cv2.putText(frame, f"{round(confidence,2)} {label}", (int(x1), int(y1)-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)

    plt.figure(figsize=(100, 100))
    plt.imshow(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))
    plt.show()  # แสดงผลลัพธ์
    plt.close()  # ปิด figure หลังแสดงผลเสร็จ